# Expose explicitly missing values with `complete`

In [1]:
import pandas as pd
import numpy as np 
import janitor
from janitor.functions.complete import  _computations_complete, _generic_complete

In [90]:
# from http://imachordata.com/2016/02/05/you-complete-me/
df = pd.DataFrame(
        {
            "Year": [1999, 2000, 2004, 1999, 2004],
            "Taxon": [
                "Saccharina",
                "Saccharina",
                "Saccharina",
                "Agarum",
                "Agarum",
            ],
            "Abundance": [4, 5, 2, 1, 8],
        }
    )

df

,Year,Taxon,Abundance
0,1999,Saccharina,4
1,2000,Saccharina,5
2,2004,Saccharina,2
3,1999,Agarum,1
4,2004,Agarum,8


In [93]:
df = pd.DataFrame(
        {
            "group": [2, 1, 1],
            "item_id": [2, 1, 2],
            "item_name": ["b", "a", "b"],
            "value1": [2, 1, 3],
            "value2": [5, 4, 6],
        }
    )
    
df

,group,item_id,item_name,value1,value2
0,2,2,b,2,5
1,1,1,a,1,4
2,1,2,b,3,6


In [ ]:
(df.)

In [25]:
df.complete('Year', 'Taxon', sort = True)

,Year,Taxon,Abundance
0,1999,Agarum,1.0
1,1999,Saccharina,4.0
2,2000,Agarum,NaN
3,2000,Saccharina,5.0
4,2004,Agarum,8.0
5,2004,Saccharina,2.0


Note that Year 2000 and Agarum pairing is missing in the DataFrame above. Let’s make it explicit:

In [26]:
# A better viewing based on order
df.complete('Year', 'Taxon', sort = False)

,Year,Taxon,Abundance
0,1999,Saccharina,4.0
1,1999,Agarum,1.0
2,2000,Saccharina,5.0
3,2000,Agarum,NaN
4,2004,Saccharina,2.0
5,2004,Agarum,8.0


In [61]:
df.complete(
        {"Year": lambda x: range(x.min(), x.max() + 1)},
        ("Taxon", "Abundance"),
        sort=True,
    )

,Year,Taxon,Abundance
0,1999,Agarum,1
1,1999,Agarum,8
2,1999,Saccharina,2
3,1999,Saccharina,4
4,1999,Saccharina,5
5,2000,Agarum,1
6,2000,Agarum,8
7,2000,Saccharina,2
8,2000,Saccharina,4
9,2000,Saccharina,5


In [84]:
(df
.set_index(['Year', 'Taxon', 'Abundance'])
.assign(dummy = 1)
# .unstack(['Taxon', 'Abundance'])
 .unstack('Year')
 .droplevel(0,1)
 .reindex(columns=range(1999,2005))
 .stack(dropna=False)
 .reset_index()
 .iloc[:, :-1]
 .reindex(columns = ['Year', 'Taxon', 'Abundance'])
 .sort_values(['Year', 'Taxon', 'Abundance'])
# .stack(['Taxon', 'Abundance'], dropna=False)
# .fillna(0, downcast='infer')
# .reset_index()
# .drop(columns='dummy')
# #.droplevel(0, 1)
)

,Year,Taxon,Abundance
0,1999,Agarum,1
6,1999,Agarum,8
12,1999,Saccharina,2
18,1999,Saccharina,4
24,1999,Saccharina,5
1,2000,Agarum,1
7,2000,Agarum,8
13,2000,Saccharina,2
19,2000,Saccharina,4
25,2000,Saccharina,5


In [5]:
%load_ext line_profiler

In [6]:
%lprun -f _generic_complete _generic_complete(df, ['Year', 'Taxon'], True, sort=True)

Timer unit: 1e-06 s

Total time: 0.008245 s
File: /workspaces/pyjanitor/janitor/functions/complete.py
Function: _generic_complete at line 268

Line #      Hits         Time  Per Hit   % Time  Line Contents
   268                                           def _generic_complete(df: pd.DataFrame, columns: list, all_strings: bool, sort: bool):
   269                                               """
   270                                               Generate cartesian product for `_computations_complete`.
   271                                           
   272                                               Returns a Series or DataFrame, with no duplicates.
   273                                               """
   274         1         11.0     11.0      0.1      if all_strings:
   275         1         10.0     10.0      0.1          if sort:
   276         1         10.0     10.0      0.1              uniques = {}
   277         3         27.0      9.0      0.3              for col in

In [7]:
ind = pd.MultiIndex.from_product([df.Year.unique(), df.Taxon.unique()], names = ['Year', 'Taxon'])
df.set_index(['Year', 'Taxon']).reindex(ind).reset_index()

,Year,Taxon,Abundance
0,1999,Saccharina,4.0
1,1999,Agarum,1.0
2,2000,Saccharina,5.0
3,2000,Agarum,NaN
4,2004,Saccharina,2.0
5,2004,Agarum,8.0


In [8]:
%lprun -f  _computations_complete  _computations_complete(df, columns=['Year', 'Taxon'], sort = True, by = None, fill_value = None, explicit = False)

Timer unit: 1e-06 s

Total time: 0.012309 s
File: /workspaces/pyjanitor/janitor/functions/complete.py
Function: _computations_complete at line 173

Line #      Hits         Time  Per Hit   % Time  Line Contents
   173                                           def _computations_complete(
   174                                               df: pd.DataFrame,
   175                                               columns: List[Union[List, Tuple, Dict, str]],
   176                                               sort: bool,
   177                                               by: Optional[Union[list, str]],
   178                                               fill_value: Optional[Union[Dict, Any]],
   179                                               explicit: bool,
   180                                           ) -> pd.DataFrame:
   181                                               """
   182                                               This function computes the final output for the `com

What if we wanted the explicit missing values for all the years from 1999 to 2004? Easy - simply pass a dictionary pairing the column name with the new values:

In [9]:
df

,Year,Taxon,Abundance
0,1999,Saccharina,4
1,2000,Saccharina,5
2,2004,Saccharina,2
3,1999,Agarum,1
4,2004,Agarum,8


In [10]:
new_year_values = {'Year': range(df.Year.min(), df.Year.max() + 1)}

df.complete(new_year_values, "Taxon")

,Year,Taxon,Abundance
0,1999,Saccharina,4.0
1,1999,Agarum,1.0
2,2000,Saccharina,5.0
3,2000,Agarum,NaN
4,2001,Saccharina,NaN
5,2001,Agarum,NaN
6,2002,Saccharina,NaN
7,2002,Agarum,NaN
8,2003,Saccharina,NaN
9,2003,Agarum,NaN


You can pass a callable as values in the dictionary:

In [11]:
new_year_values = lambda year: range(year.min(), year.max() + 1)

df.complete({"Year": new_year_values}, "Taxon", sort = True)

,Year,Taxon,Abundance
0,1999,Agarum,1.0
1,1999,Saccharina,4.0
2,2000,Agarum,NaN
3,2000,Saccharina,5.0
4,2001,Agarum,NaN
5,2001,Saccharina,NaN
6,2002,Agarum,NaN
7,2002,Saccharina,NaN
8,2003,Agarum,NaN
9,2003,Saccharina,NaN


You can get explcit rows, based only on existing data:

In [12]:
# https://stackoverflow.com/q/62266057/7175713
df = {"Name" : ("Bob", "Bob", "Emma"), 
       "Age" : (23,23,78), 
       "Gender" :("Male", "Male", "Female"), 
       "Item" : ("house", "car", "house"), 
       "Value" : (5,1,3)
        }
df = pd.DataFrame(df)
df

,Name,Age,Gender,Item,Value
0,Bob,23,Male,house,5
1,Bob,23,Male,car,1
2,Emma,78,Female,house,3


In the DataFrame above, there is no `car` Item value for the `Name`, `Age`, `Gender`  combination -> `(Emma, 78, Female)`. Pass `(Name, Age, Gender)` and `Item` to explicitly expose the missing row:

In [13]:
df.complete(('Name', 'Age', 'Gender'), 'Item')

,Name,Age,Gender,Item,Value
0,Bob,23,Male,house,5.0
1,Bob,23,Male,car,1.0
2,Emma,78,Female,house,3.0
3,Emma,78,Female,car,NaN


The example above showed how to expose missing rows on a group basis. There is also the option of exposing missing rows with the `by` parameter: 

In [85]:
df = pd.DataFrame(
        {
            "state": ["CA", "CA", "HI", "HI", "HI", "NY", "NY"],
            "year": [2010, 2013, 2010, 2012, 2016, 2009, 2013],
            "value": [1, 3, 1, 2, 3, 2, 5],
        }
    )

df

,state,year,value
0,CA,2010,1
1,CA,2013,3
2,HI,2010,1
3,HI,2012,2
4,HI,2016,3
5,NY,2009,2
6,NY,2013,5


In [89]:
df.set_index('year').groupby('state').apply(lambda x: x.reindex(range(x.index.min(), x.index.max() + 1))).drop(columns='state').reset_index()

,state,year,value
0,CA,2010,1.0
1,CA,2011,NaN
2,CA,2012,NaN
3,CA,2013,3.0
4,HI,2010,1.0
5,HI,2011,NaN
6,HI,2012,2.0
7,HI,2013,NaN
8,HI,2014,NaN
9,HI,2015,NaN


Let's expose all the missing years, based on the minimum and maximum year, for each state:

In [15]:
new_year_values = lambda year: range(year.min(), year.max() + 1)

result = df.complete(
            {'year': new_year_values},
            by='state',
            sort = False
        )

result

,state,year,value
0,CA,2010,1.0
1,CA,2011,NaN
2,CA,2012,NaN
3,CA,2013,3.0
4,HI,2010,1.0
5,HI,2011,NaN
6,HI,2012,2.0
7,HI,2013,NaN
8,HI,2014,NaN
9,HI,2015,NaN


You can fill the nulls with Pandas' `fillna`:

In [16]:
result.fillna(0, downcast = 'infer')

,state,year,value
0,CA,2010,1
1,CA,2011,0
2,CA,2012,0
3,CA,2013,3
4,HI,2010,1
5,HI,2011,0
6,HI,2012,2
7,HI,2013,0
8,HI,2014,0
9,HI,2015,0


In [46]:
df = pd.DataFrame(
        dict(
            group=(1, 2, 1, 2),
            item_id=(1, 2, 2, 3),
            item_name=("a", "a", "b", "b"),
            value1=(1, np.nan, 3, 4),
            value2=range(4, 8),
        )
    )

df = df.astype({'value1':'category'})

In [47]:
df['value1'] = df['value1'].cat.add_categories([0])

(df
.set_index(['group', 'item_id', 'item_name'])
.unstack('group', fill_value = 0)
 .stack(dropna=False)
 .reset_index()
 .reindex(columns=df.columns)
 .sort_values(['group', 'item_id', 'item_name'])
).dtypes

group         int64
item_id       int64
item_name    object
value1       object
value2        int64
dtype: object

In [18]:
print(df.dtypes)

group           int64
item_id         int64
item_name      object
value1       category
value2          int64
dtype: object


In [22]:
df.complete(
        "group",
        ("item_id", "item_name"),
        sort=False,
    )

,group,item_id,item_name,value1,value2
0,1,1,a,1.0,4.0
1,1,2,a,NaN,NaN
2,1,2,b,3.0,6.0
3,1,3,b,NaN,NaN
4,2,1,a,NaN,NaN
5,2,2,a,NaN,5.0
6,2,2,b,NaN,NaN
7,2,3,b,4.0,7.0


In [23]:
df.complete(
        "group",
        ("item_id", "item_name"),
        sort=True,
    )

,group,item_id,item_name,value1,value2
0,1,1,a,1.0,4.0
1,1,2,a,NaN,NaN
2,1,2,b,3.0,6.0
3,1,3,b,NaN,NaN
4,2,1,a,NaN,NaN
5,2,2,a,NaN,5.0
6,2,2,b,NaN,NaN
7,2,3,b,4.0,7.0


In [20]:
df.complete(
        "group",
        ("item_id", "item_name"),
        fill_value=0,
        explicit=False,
    )

,group,item_id,item_name,value1,value2
0,1,1,a,1.0,4.0
1,1,2,a,0.0,0.0
2,1,2,b,3.0,6.0
3,1,3,b,0.0,0.0
4,2,1,a,0.0,0.0
5,2,2,a,NaN,5.0
6,2,2,b,0.0,0.0
7,2,3,b,4.0,7.0


In [21]:
df

,group,item_id,item_name,value1,value2
0,1,1,a,1.0,4
1,2,2,a,NaN,5
2,1,2,b,3.0,6
3,2,3,b,4.0,7
